In [3]:
import os

import numpy as np
import pandas as pd
import networkx as nx
import community as community_louvain
from sklearn.feature_extraction.text import CountVectorizer
from itertools import combinations
from scipy.spatial import distance

import string
from zipfile import ZipFile
import pandas as pd
import itertools
# nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk import tokenize, pos_tag

s=set(stopwords.words('english'))
start = 'C:/Users/gony/Desktop/data/'

Functions:

In [5]:
def remove_stopwords_numbers(text):
    return list(filter(lambda w: not w.lower() in s and not w.isdigit(),text))


def calc_top_clusters(G):
    clustering_coefficients = nx.clustering(G)
    partition = community_louvain.best_partition(G)
    # Iterate over the partition dictionary
    cluster_nodes = {}
    for node, community_id in partition.items():
        if community_id not in cluster_nodes:
            cluster_nodes[community_id] = [node]
        else:
            cluster_nodes[community_id].append(node)
    cluster_dict = {}
    for word, cluster in partition.items():
      if cluster in cluster_dict.keys():
        cluster_dict[cluster].append(word)
      else:
        cluster_dict[cluster] = [word]

    cluster_size = {}
    for key in cluster_dict.keys():
      cluster_size[key] = len(cluster_dict[key])


    # Sort the clusters based on their size in descending order
    sorted_clusters = sorted(cluster_size.items(), key=lambda x: x[1], reverse=True)

    # Print the top clusters based on their size
    top_cluster = sorted_clusters[:20]

    return top_cluster, cluster_dict, clustering_coefficients

def find_cluster(graph, node):
    """
    Finds the cluster of a given node in a NetworkX graph.
    Args:
        graph (nx.Graph or nx.DiGraph): The NetworkX graph.
        node: The node for which to find the cluster.
    Returns:
        set: A set containing the nodes in the cluster.
    """
    # Find all connected components
    components = list(nx.connected_components(graph))

    # Find the component that contains the node
    for component in components:
        if node in component:
            return component

    # If the node is not found, return an empty set
    return set()

def find_communities(graph):
    # Apply the Louvain algorithm to detect communities
    partition = community_louvain.best_partition(graph)

    # Create a dictionary to store communities
    communities = {}

    # Iterate over nodes and their corresponding communities
    for node, community_id in partition.items():
        if community_id not in communities:
            communities[community_id] = []
        communities[community_id].append(node)

    # Calculate the community density for each community
    community_density = {}
    for community_id, community in communities.items():
        community_subgraph = graph.subgraph(community)
        internal_edges = community_subgraph.number_of_edges()
        total_edges = nx.density(community_subgraph) * nx.number_of_edges(community_subgraph)
        if total_edges==0:
            print(community_id)
            community_density[community_id] = 0
        else:
            community_density[community_id] = internal_edges / total_edges

    # Return the communities and their density
    return list(communities.values()), community_density

# Boston Marathon Bombing:

In [44]:
path = start + 'nodes - Copy/boston_15_04_2013/'
# for f in os.listdir(path):!pip uninstall networkx

#     day = pd.read_csv(path+f)
#     G = nx.from_pandas_edgelist(day, 'source', 'target', 'weight')
#     top_cluster, cluster_dict = calc_top_clusters(G)

day = pd.read_csv(path+'nodes16.04.2013.csv')
G = nx.from_pandas_edgelist(day, 'source', 'target', 'weight')
# top_cluster, cluster_dict, clustering_coefficients = calc_top_clusters(G)
# nodes_in_clustet = find_cluster(G, 'marathon')

Community density measures the density of connections within a community compared to the density of connections outside the community.

In [45]:
communities, community_density = find_communities(G)

communities_df = pd.DataFrame(columns=['id', 'nodes', 'density'])
for i, community in enumerate(communities):
    communities_df.loc[len(communities_df)] = [i+1, community, community_density[i]]

121
143
175
187
194
196


In [71]:
communities_df.head()
from tqdm import tqdm

In [81]:
communities_df.sort_values(['density'], ascending=False)

centrality = pd.read_csv(start+'nodes_centrality/boston_15_04_2013/centrality_16.04.2013.csv')
top_degree = []
for i, row in communities_df.iterrows():
    nodes = row['nodes']
    temp = pd.DataFrame(columns=['nodes', 'deg_cent'])
    for w in nodes:
        if w in centrality['nodes']:
            temp.loc[len(temp)] = [w, centrality[centrality['nodes']==w]['degree'].values[0]]
        else:
            temp.loc[len(temp)] = [w, 0]
    temp.sort_values(['deg_cent'], ascending=False, inplace=True)
    top_degree.append(temp[:10].nodes.values)

communities_df['top_degree_cent'] = top_degree

100%|██████████| 2/2 [00:00<00:00, 166.67it/s]


In [84]:
communities_df.sort_values(['density'], ascending=False, inplace=True)
communities_df.head()

,id,nodes,density,top_degree_cent
6,7,"[1800call, encourage, cheer, affiliate, crowd,...",39.000000,"[1800call, invariably, eternity, kingdom, herm..."
10,11,"[26timestwo, perfectly, accurate, conduct, swe...",37.752475,"[26timestwo, reliably, methodically, math, mar..."
38,39,"[acquire, ingredient, variety, blur, airborne,...",37.033898,"[acquire, hearse, emphasize, broader, screenin..."
43,44,"[afar, heartbroken, hometown, native, alongsid...",34.140394,"[afar, hardly, infrastructure, heartfelt, hear..."
61,62,"[babysat, babysit]",30.781609,"[babysat, babysit]"


In [87]:
communities_df[['id', 'top_degree_cent']][:10]

,id,top_degree_cent
6,7,"[1800call, invariably, eternity, kingdom, herm..."
10,11,"[26timestwo, reliably, methodically, math, mar..."
38,39,"[acquire, hearse, emphasize, broader, screenin..."
43,44,"[afar, hardly, infrastructure, heartfelt, hear..."
61,62,"[babysat, babysit]"
58,59,"[attractive, glowing]"
47,48,"[alto, sax, lovely]"
20,21,"[abandon, sneaker, stroller]"
15,16,"[40s, 70, contends, print, finger, tends, hall..."
11,12,"[29yearold, babysitting, hardest, pass, invite..."
